In [1]:
import numpy as np
import tensorflow as tf

def sigmoid(x):
    return 1/(1+np.exp(-x))

In [2]:
data_x1 = np.array([[0., 1.], [1, 0], [1, 1], [0, -1], [-1, 0], [-1, -1], [-1,-1]])
data_x2 = np.array([[1., 0.], [-1, 1], [0, -2], [2, 1], [1, -1], [2, 2], [1, 1]])
data_xx = np.array([data_x1, data_x2])

n_inputs = data_x1.shape[0]
n_nodes = 3
n_steps = 2

state_0 = np.zeros([n_inputs, n_nodes])
state_0

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

sigmoind(Wx + Uy + b)

In [3]:
W = np.array([[1, 0, -1], [2, 2, 1]])
U = np.array([[1, 1, 0], [0, -2, -1], [-1, 0, 1]])
b = np.array([0.5, -0.5, 1])

WU = np.concatenate([W, U])
WU

array([[ 1,  0, -1],
       [ 2,  2,  1],
       [ 1,  1,  0],
       [ 0, -2, -1],
       [-1,  0,  1]])

In [4]:
def getState(data_x, state, WU, b):
    data = np.concatenate([data_x, state], axis=1)
    return sigmoid(np.matmul(data, WU) + b)

state_1 = getState(data_x1, state_0, WU, b)
state_2 = getState(data_x2, state_1, WU, b)

In [5]:
state_1

array([[ 0.92414182,  0.81757448,  0.88079708],
       [ 0.81757448,  0.37754067,  0.5       ],
       [ 0.97068777,  0.81757448,  0.73105858],
       [ 0.18242552,  0.07585818,  0.5       ],
       [ 0.37754067,  0.37754067,  0.88079708],
       [ 0.07585818,  0.07585818,  0.73105858],
       [ 0.07585818,  0.07585818,  0.73105858]])

In [6]:
state_2

array([[ 0.82395043,  0.2295229 ,  0.51580039],
       [ 0.86027483,  0.82671081,  0.95780972],
       [ 0.03695601,  0.00568365,  0.25227493],
       [ 0.98496796,  0.82211005,  0.60447392],
       [ 0.26830165,  0.05327504,  0.37830624],
       [ 0.99711344,  0.96845057,  0.83959266],
       [ 0.94504924,  0.8059869 ,  0.83959266]])

In [7]:
inputs = tf.placeholder(tf.float32, [n_steps, n_inputs, 2])
inputs_seq = tf.unstack(inputs)
with tf.name_scope('rec_layer'):
    cell = tf.contrib.rnn.core_rnn_cell.BasicRNNCell(num_units=3, activation=tf.sigmoid)
    output, state = tf.contrib.rnn.static_rnn(
        cell=cell, inputs=inputs_seq, dtype=tf.float32)
    

In [9]:
cell_weight = [v for v in tf.trainable_variables() if v.name == 'rnn/basic_rnn_cell/weights:0'][0]
update_weight = tf.assign(cell_weight, WU)

In [10]:
cell_bias = [v for v in tf.trainable_variables() if v.name == 'rnn/basic_rnn_cell/biases:0'][0]
update_bias = tf.assign(cell_bias, b)

In [20]:
feed_dict = {inputs: data_xx}

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    variables_names =[v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    print('assigned values')
    for k,v in zip(variables_names, values):
        print(k, v)
    sess.run(update_weight)
    sess.run(update_bias)
    print('updated')
    for k,v in zip(variables_names, values):
        print(k, v)
    out, st = sess.run((output, state), feed_dict)
    print('output1')
    print(out[0])
    print('output2')
    print(out[1])
    print('state')
    print(st)

assigned values
rnn/basic_rnn_cell/weights:0 [[-0.30108619 -0.80872941  0.72268444]
 [-0.78550947  0.1273762  -0.71344316]
 [ 0.45219082 -0.47534493 -0.46869576]
 [-0.03871292 -0.53958845 -0.50132215]
 [ 0.02187312  0.09094775 -0.78896606]]
rnn/basic_rnn_cell/biases:0 [ 0.  0.  0.]
updated
rnn/basic_rnn_cell/weights:0 [[-0.30108619 -0.80872941  0.72268444]
 [-0.78550947  0.1273762  -0.71344316]
 [ 0.45219082 -0.47534493 -0.46869576]
 [-0.03871292 -0.53958845 -0.50132215]
 [ 0.02187312  0.09094775 -0.78896606]]
rnn/basic_rnn_cell/biases:0 [ 0.  0.  0.]
output1
[[ 0.92414182  0.81757444  0.88079703]
 [ 0.81757444  0.37754068  0.5       ]
 [ 0.97068775  0.81757444  0.7310586 ]
 [ 0.18242553  0.07585818  0.5       ]
 [ 0.37754068  0.37754068  0.88079703]
 [ 0.07585818  0.07585818  0.7310586 ]
 [ 0.07585818  0.07585818  0.7310586 ]]
output2
[[ 0.82395041  0.22952288  0.51580036]
 [ 0.86027485  0.82671082  0.95780975]
 [ 0.03695601  0.00568365  0.25227493]
 [ 0.98496801  0.82211006  0.604473